## Scraping video data using YouTube API

#### Set-up

Run the following instructions on your terminal:
``` bash
pip install google-api-python-client
pip install --upgrade google-api-python-client
```

In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

import os
os.chdir(os.path.expanduser("../"))

from dees_package.youtube_functions import *

print("Current working directory:", os.getcwd())

Current working directory: /Users/hanbinfeng/Desktop/LSE_Data_Science/ds105a-project-dees-nuts


In [2]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [3]:
# creating service object of the youtube version 3 API
service_youtube = build('youtube', 'v3', developerKey=credentials['youtube_api_key'])

## Getting a list of music videos so that we have a list of videos to analyse


Carry out test search using `youtube_search`

In [4]:
youtube_search_data, video_id = youtube_search(service_youtube, 2000, "official music video", "video", "US", 10)

with open('./data/yt_search_data.json', 'w') as json_file:
    json.dump(youtube_search_data, json_file, indent=4)

yt_search_df = pd.DataFrame(youtube_search_data)


1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000


In [5]:
len(video_id)

633

In [6]:
# check = pd.read_csv('./data/merged.csv')

# type(check['like_count'][0])

In [7]:
yt_search_df.to_csv('./data/search.csv')

## From the list of music videos, get statistics on each video


In [8]:
video_stats = get_stats(service_youtube, video_id)  # there is a limit on the number of video ids, can only run 50 at a time. Solution: create different lists with 50 IDs each.

#with open('./data/mv_stats_data.json', 'w') as json_file:
#     json.dump(video_stats, json_file, indent=4)

#videostats_df = pd.DataFrame(video_stats)

0 49
[{'kind': 'youtube#video', 'etag': '3p6v4f1vZ5Hg5UFck7AJLgn39po', 'id': 'JGwWNGJdvx8', 'statistics': {'viewCount': '6186367395', 'likeCount': '32532410', 'favoriteCount': '0', 'commentCount': '1150108'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': 'RwbccSZJPm2Xlz9Re7kar4ahOWM', 'id': '09R8_2nJtjg', 'statistics': {'viewCount': '3996159437', 'likeCount': '15981340', 'favoriteCount': '0', 'commentCount': '420729'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': 'p7EIUv84adpZnWzUJ9-t3rfqG1Y', 'id': 'pRpeEdMmmQ0', 'statistics': {'viewCount': '3839678587', 'likeCount': '22118344', 'favoriteCount': '0', 'commentCount': '1321846'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Music_of_Latin_America', 'https://e

[{'kind': 'youtube#video', 'etag': 'JX0MhFiA29OADxtU_fTuTB7iLNw', 'id': 'bKDdT_nyP54', 'statistics': {'viewCount': '1177735631', 'likeCount': '8360208', 'favoriteCount': '0', 'commentCount': '202974'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Hip_hop_music', 'https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music', 'https://en.wikipedia.org/wiki/Rhythm_and_blues', 'https://en.wikipedia.org/wiki/Soul_music']}}, {'kind': 'youtube#video', 'etag': 'bDlkY_zX8ml9HF3SBwUu5j6yAe8', 'id': 'SR6iYWJxHqs', 'statistics': {'viewCount': '1176478250', 'likeCount': '5732756', 'favoriteCount': '0', 'commentCount': '303235'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': 'S0Px9QER8ylvPanOsuZAVUfYYf4', 'id': '4LfJnj66HVQ', 'statistics': {'viewCount': '1173995160', 'likeCount': '12314516', 'favoriteCount': '0', 'commentCount': '758927'}, 'topicDeta

In [9]:
video_stats_df = pd.DataFrame(video_stats)

In [19]:
# merge the mv stats and search dataframes
merged_df = pd.merge(yt_search_df, video_stats_df, left_on='video_id', right_on='video_id')
merged_df.to_json('./data/merged.json')

merged_df.to_csv('./data/merged.csv')

In [20]:
comments_df = get_comments_in_videos(service_youtube, video_id) # note that comments are disabled for some videos

KeyboardInterrupt: 

In [21]:
comments_df.head()

,video_id,comments
0,JGwWNGJdvx8,[Share this if you like songs https://youtu.be...
1,09R8_2nJtjg,"[Me\n😊, Me I see in 2024, Who's watching this ..."
2,pRpeEdMmmQ0,"[🎉🎉🤩🤩🤩shakira eres la mejor ❤❤ alguien 2024, 2..."
3,lp-EO5I60KA,[Y did this get so many views? It's not even c...
4,2Vv-BfVoq4g,"[01-26-2024, Simplesmente te amo. Amo imensame..."


In [1]:
deets_df = get_video_details(service_youtube, video_id)

NameError: name 'get_video_details' is not defined

In [14]:
merged_df

,video_id,title,channel_id,channel_title,view_count,comment_count,wikipedia_categories
0,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://..."
1,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://..."
2,SSLPIT5Hi5w,Aidonia - Bottles | Official Music Video,UCG2QFlHjY3yAPLXbzH3_vSQ,AidoniaVEVO,237034,1642,[https://en.wikipedia.org/wiki/Electronic_musi...
3,SSLPIT5Hi5w,Aidonia - Bottles | Official Music Video,UCG2QFlHjY3yAPLXbzH3_vSQ,AidoniaVEVO,237034,1642,[https://en.wikipedia.org/wiki/Electronic_musi...
4,vVLl0xI_qD0,Saweetie - IMMORTAL FREESTYLE (Official Music ...,UC2FCUwThC0RAzaQoI0TtpLA,Official Saweetie,240727,1802,"[https://en.wikipedia.org/wiki/Hip_hop_music, ..."
5,1g_fHPyC974,Punsang Appun // Teaser Brishtisikha // Banmal...,UC_I_VcN-MM9TtMXepCYUqYg,STUDIO OBONORI,232399,64,"[https://en.wikipedia.org/wiki/Music, https://..."
6,l0_OFC10hTE,Guns N&#39; Roses - The General (Official Musi...,UCIak6JLVOjqhStxrL1Lcytw,Guns N' Roses,225220,1851,"[https://en.wikipedia.org/wiki/Music, https://..."
7,AnnRwkxi4mg,Joseph Attieh - Hadder Halak [Official Music V...,UCqEQp8OWk2p2fWBZjyscZUA,Joseph Attieh,223891,366,[https://en.wikipedia.org/wiki/Music]
8,ysftXz2INGs,Soulja - Ronaldo (Official Music Video) | سولج...,UCU-3aVH2u__JEZFCXShiRTQ,SOULJA - سولجا,237172,900,"[https://en.wikipedia.org/wiki/Hip_hop_music, ..."
9,Upi2cBoGwnM,শিকারী - Itz Kabbo (Official Music Video) 🔥 F...,UCE-8FgcXvnAsb2EG1FizSyg,Itz Kabbo,210675,4728,"[https://en.wikipedia.org/wiki/Hip_hop_music, ..."


In [15]:
comments_df

,video_id,comments
0,JGwWNGJdvx8,[Share this if you like songs https://youtu.be...
1,09R8_2nJtjg,"[Me\n😊, Me I see in 2024, Who's watching this ..."
2,pRpeEdMmmQ0,"[🎉🎉🤩🤩🤩shakira eres la mejor ❤❤ alguien 2024, 2..."
3,lp-EO5I60KA,[Y did this get so many views? It's not even c...
4,2Vv-BfVoq4g,"[01-26-2024, Simplesmente te amo. Amo imensame..."
...,...,...
624,c91I3oNe-KE,[semoga keng bapak enggal dhangan]
625,u2lcTDjjdOg,[so underrated...]
626,ctgiOmgBw6Q,[初の弾き語り曲です✨世界の誰かの心の安らぎになれば嬉しいですぜひご覧ください💕💕またご協力...
627,zyvjriX3Dag,"[boor and quanny hold in philly down, 🤧🤧, 🔥🔥🔥,..."


In [25]:

final_youtube_df = pd.merge(pd.merge(merged_df,deets_df,left_on='video_id', right_on='video_id', sort = False, ), comments_df, on='video_id')
final_youtube_df.to_csv('./data/final_youtube.csv')
final_youtube_df.head(100)

,video_id,title_x,channel_id,channel_title,view_count,comment_count,wikipedia_categories,channelTitle,title_y,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,comments
0,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://...",(G)I-DLE (여자)아이들 (Official YouTube Channel),#Wife Official Music Video,,None,2024-01-22T03:09:25Z,243350,36444,None,314,PT53S,hd,false,"[Anaconda?, This Beauties is it a joke?😍😍🔥, Кр..."
1,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://...",(G)I-DLE (여자)아이들 (Official YouTube Channel),#Wife Official Music Video,,None,2024-01-22T03:09:25Z,243350,36444,None,314,PT53S,hd,false,"[Anaconda?, This Beauties is it a joke?😍😍🔥, Кр..."
2,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://...",(G)I-DLE (여자)아이들 (Official YouTube Channel),#Wife Official Music Video,,None,2024-01-22T03:09:25Z,243350,36444,None,314,PT53S,hd,false,"[Anaconda?, This Beauties is it a joke?😍😍🔥, Кр..."
3,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://...",(G)I-DLE (여자)아이들 (Official YouTube Channel),#Wife Official Music Video,,None,2024-01-22T03:09:25Z,243350,36444,None,314,PT53S,hd,false,"[Anaconda?, This Beauties is it a joke?😍😍🔥, Кр..."
4,1oUIm6vEGgw,#Wife Official Music Video,UCritGVo7pLJLUS8wEu32vow,(G)I-DLE (여자)아이들 (Official YouTube Channel),243350,314,"[https://en.wikipedia.org/wiki/Music, https://...",(G)I-DLE (여자)아이들 (Official YouTube Channel),#Wife Official Music Video,,None,2024-01-22T03:09:25Z,243350,36444,None,314,PT53S,hd,false,"[Anaconda?, This Beauties is it a joke?😍😍🔥, Кр..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,IRDSMDpg3Rc,Ruszó Tibi - Imát mondok (Official Music Video),UCdQBirZriN5T_Jct_Dgbnvg,TopManagement,18688,43,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",TopManagement,Ruszó Tibi - Imát mondok (Official Music Video),"Üzleti megkeresés, fellépésszervezés: \nLantos...","[bottka, m.k, bottka x m.k, ruszó tibi, szívem...",2024-01-25T16:00:07Z,18688,1054,None,43,PT2M28S,hd,false,"[Nagyon -nagyon ❤❤❤❤, Nagyon királylett, Ez eg..."
73,IRDSMDpg3Rc,Ruszó Tibi - Imát mondok (Official Music Video),UCdQBirZriN5T_Jct_Dgbnvg,TopManagement,18688,43,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",TopManagement,Ruszó Tibi - Imát mondok (Official Music Video),"Üzleti megkeresés, fellépésszervezés: \nLantos...","[bottka, m.k, bottka x m.k, ruszó tibi, szívem...",2024-01-25T16:00:07Z,18688,1054,None,43,PT2M28S,hd,false,"[Nagyon -nagyon ❤❤❤❤, Nagyon királylett, Ez eg..."
74,IRDSMDpg3Rc,Ruszó Tibi - Imát mondok (Official Music Video),UCdQBirZriN5T_Jct_Dgbnvg,TopManagement,18688,43,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",TopManagement,Ruszó Tibi - Imát mondok (Official Music Video),"Üzleti megkeresés, fellépésszervezés: \nLantos...","[bottka, m.k, bottka x m.k, ruszó tibi, szívem...",2024-01-25T16:00:07Z,18688,1054,None,43,PT2M28S,hd,false,"[Nagyon -nagyon ❤❤❤❤, Nagyon királylett, Ez eg..."
75,IRDSMDpg3Rc,Ruszó Tibi - Imát mondok (Official Music Video),UCdQBirZriN5T_Jct_Dgbnvg,TopManagement,18688,43,"[https://en.wikipedia.org/wiki/Hip_hop_music, ...",TopManagement,Ruszó Tibi - Imát mondok (Official Music Video),"Üzleti megkeresés, fellépésszervezés: \nLantos...","[bottka, m.k, bottka x m.k, ruszó tibi, szívem...",2024-01-25T16:00:07Z,18688,1054,None,43,PT2M28S,hd,false,"[Nagyon -nagyon ❤❤❤❤, Nagyon királylett, Ez eg..."
